In [1]:
import re
import os
import yaml
import json
import pandas as pd
from google.cloud import bigquery
import google.api_core.exceptions as google_exceptions

In [2]:
config  = yaml.safe_load(open('config.yaml', 'r'))

In [3]:
query_dir = 'queries'

In [4]:
queries = {}
for query_file in (os.listdir(query_dir)):
    with open(os.path.join(query_dir, query_file), 'r') as query:
        queries[query_file] = query.read()

In [5]:
query_client = bigquery.Client()

In [6]:
def exists_table(table_reference, client):
    try:
        client.get_table(table_reference)
        return True
    except google_exceptions.NotFound:
        return False

In [7]:
if exists_table(f'{config["project_id"]}.{config["dataset"]}.expanded_sub_properties', query_client):
    request = queries['all_new_buildings_that_have_not_been_expanded.sql']
else:
    request = queries['all_new_buildings.sql']

In [8]:
real_estate_raw = query_client.query(request).to_dataframe()

In [9]:
real_estate_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
ad_id             103 non-null int64
new_building      103 non-null bool
property_table    103 non-null object
property_attrs    103 non-null object
address           103 non-null object
dtypes: bool(1), int64(1), object(3)
memory usage: 3.4+ KB


In [10]:
real_estate_raw.set_index(keys='ad_id', drop=True, inplace=True)

In [11]:
real_estate_raw.head()

,new_building,property_table,property_attrs,address
ad_id,,,,
71962400,True,"{""Bolig"": [""E-301""], ""Soverom"": [""3""], ""Etg"": ...","{""Areal"": ""129 m\u00b2"", ""Soverom"": ""3-3"", ""Bo...","Fjellhamar Torg, 1472 Fjellhamar"
76674446,True,"{""Bolig"": [""D1302"", ""D1303"", ""D1402"", ""D1503"",...","{""Fellesgjeld"": ""2\u00a0500\u00a0000 \u2013 3\...","Landstadsgate 13, 2004 Lillestrøm"
77962792,True,"{""Bolig"": [""21"", ""27"", ""28"", ""29"", ""30"", ""32"",...","{""Omkostninger"": ""22\u00a0665 kr"", ""Totalpris""...","Gamle Lommedalsvei 123, 1348 Rykkinn"
83008842,True,"{""Bolig"": [""111""], ""Soverom"": [""3""], ""Etg"": [""...","{""Areal"": ""68 m\u00b2"", ""Soverom"": ""3-3"", ""Bol...","Mørtelverksbakken 27, 0580 Oslo"
87990816,True,"{""Bolig"": [""10001"", ""10002"", ""C0003"", ""C1002"",...","{""Omkostninger"": ""22\u00a0650 kr"", ""Totalpris""...","Røakollen - Aslakveien 20 - hus C, 0753 Oslo"


In [12]:
for col in ['property_table', 'property_attrs']:
    real_estate_raw[col] = [json.loads(cell) for cell in real_estate_raw[col]]

In [13]:
def expanand_property_tables(new_buildings):
    cols = new_buildings.iloc[0]['property_table'].keys()
    col_map = {'Bolig': 'apt_id',
               'Soverom': 'num_bedrooms',
               'Etg': 'floor',
               'P-rom': 'primary_size',
               'BRA': 'total_size',
               'Totalpris': 'price',
               'Pris': 'price'}
    property_tables = pd.DataFrame(columns=cols)
    for code, row in new_buildings['property_table'].iteritems():
        all_properties = [arr for arr in row.values()]
        to_append = dict(zip(cols, all_properties))
        frame = pd.DataFrame(to_append, columns=cols)
        frame['ad_id'] = code
        property_tables = property_tables.append(frame, ignore_index=True, sort=False)
    property_tables.rename(columns=col_map, inplace=True)
    return property_tables

In [14]:
sub_properties = expanand_property_tables(real_estate_raw[(real_estate_raw.new_building == True) & (real_estate_raw.property_table)])

In [15]:
sub_properties.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,E-301,3,-,129 m²,137 m²,7 690 000 kr,71962400.0
1,D1302,2,3.,-,76.4 m²,5 031 462 kr,76674446.0
2,D1303,2,3.,-,91.3 m²,5 981 462 kr,76674446.0
3,D1402,2,4.,-,76.4 m²,5 231 462 kr,76674446.0
4,D1503,2,5.,-,89.7 m²,6 931 462 kr,76674446.0


In [16]:
def get_int_from_str(string):
    try:
        concat_string = ''.join(re.findall(r'[\d.]', string))
        if concat_string != '':
            return float(concat_string)
    except Exception as e:
        print(e, string)
        pass

In [17]:
sub_properties_parsed = sub_properties.copy()
for col in ['primary_size', 'floor', 'price', 'num_bedrooms', 'total_size']:
    sub_properties_parsed[col] = [get_int_from_str(s) for s in sub_properties_parsed[col]]

expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None


In [18]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1144 entries, 0 to 1143
Data columns (total 7 columns):
apt_id          1144 non-null object
num_bedrooms    1100 non-null float64
floor           1035 non-null float64
primary_size    1093 non-null float64
total_size      1142 non-null float64
price           1141 non-null float64
ad_id           1144 non-null float64
dtypes: float64(6), object(1)
memory usage: 62.6+ KB


In [19]:
sub_properties_parsed.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,E-301,3.0,NaN,129.0,137.0,7690000.0,71962400.0
1,D1302,2.0,3.0,NaN,76.4,5031462.0,76674446.0
2,D1303,2.0,3.0,NaN,91.3,5981462.0,76674446.0
3,D1402,2.0,4.0,NaN,76.4,5231462.0,76674446.0
4,D1503,2.0,5.0,NaN,89.7,6931462.0,76674446.0


In [20]:
sub_properties_parsed.drop_duplicates(inplace=True)

In [21]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1143 entries, 0 to 1143
Data columns (total 7 columns):
apt_id          1143 non-null object
num_bedrooms    1099 non-null float64
floor           1034 non-null float64
primary_size    1092 non-null float64
total_size      1141 non-null float64
price           1140 non-null float64
ad_id           1143 non-null float64
dtypes: float64(6), object(1)
memory usage: 71.4+ KB


In [22]:
sub_properties_parsed.to_gbq(destination_table='housing_data.expanded_sub_properties',
                             project_id='hde-test-clean',
                             if_exists='append')